# Segmentation using the qim.register module
*TODO: Figure our how to "install" the module so that I don't need to put the github folder in the path.*

## Load modules

In [1]:
import sys, os
sys.path.insert(0, '/home/hektor/scripts')
import qim.registration as registration
import tifffile
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

from skimage import filters


## Read data
Each of samples can be loaded by changing sample_number.
* sample_number = 1: The large sample at the top
* sample_number = 2: Second sample from the top
* sample_number = 3: Third sample from the top (bottom left)
* sample_number = 4: Fourth sample from the top (bottom right)
* sample_number = 5: Josefins sample

In [2]:
sample_number = 2

qimfolder = '/lunarc/nobackup/projects/qim/QIM_2020_LU_Josefin_GEO'
if sample_number == 5:
    datapath = os.path.join(qimfolder,'RegistrationILL/Bin1')
    x_data = os.path.join(datapath,'x_registered.tiff')
    n_data = os.path.join(datapath,'neutron-bin1.tif')
else:
    datapath = os.path.join(qimfolder,'johan_registration/bin1')
    x_data = os.path.join(datapath,'s{:d}_xray_b1.tif'.format(sample_number))
    n_data = os.path.join(datapath,'s{:d}_neutron_b1.tif'.format(sample_number))

x_volume = tifffile.imread(x_data)
n_volume = tifffile.imread(n_data)


## Apply median filter to the neutron data
This is to reduce the noise a bit. It might be worth playing a little bit here

In [3]:
s = 3 #this is the size of the filter
selem = np.ones((s,s,s))
n_filt = filters.median(n_volume,selem=selem)

fig,axes=plt.subplots(1,2,figsize=(8,4))
for t, volume in zip(['Original','Filtered'],[n_volume,n_filt]):
    hist,be = np.histogram(volume.flatten(),bins=100,range=(5000,volume.max()))
    bc = (be[1:] + be[:-1])/2 #bin centers (arithmetic mean)
    ax = axes.ravel()
    ax[0].hist(volume.flatten(),bins=100,range=(5000,volume.max()))
    ax[0].set_title('Linear scale')
    ax[1].semilogy(bc,hist,label=t)
    ax[1].set_title('Logarithmic scale')
    for a in ax:
        a.set_xlabel('Grayscale value')
        a.set_ylabel('Counts')
fig.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

## Create the dual histogram

The x-ray data will be on the x-axis and the neutron data on the y-axis. The number of bins will be the same on both axes.

*TODO: perhaps it is useful to have different number of bins on the axes*


In [4]:
nbins = 256
H,xcen,ycen = registration.make_dual_histogram(x_volume,n_filt,nbins)

#make a plot
registration.plot_dual_histogram(H,xcen,ycen,xlabel='X-ray intensity',ylabel='Neutron intensity',vmax=1500)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

## Fit Gaussians to the histogram
This will automatically look for maximas of the histogram.

*TODO: Make it work better for peaks at the edges*


In [5]:
# generate markers
markers = registration.setup_markers(H,xcen,ycen)
# Fit a Gaussian to each marker
opts = registration.fit_gaussians(H,markers,xcen,ycen)
# Make a plot with markers and Gaussians
registration.plot_dual_histogram(H,xcen,ycen,xlabel='X-ray intensity',ylabel='Neutron intensity',vmax=1500,markers=markers,fits=opts)

Trying to fit peak at (45695.3, 28753.2)
Initial guess: [45695.303 28753.186 1000.0 1000.0 0.0 17127.000000 0.0]
Fitted parameters [45321.804 28998.006 1932.9 3142.8 -0.0 16130.172961 0.0]
Fitted 1 peaks
Trying to fit peak at (38271.4, 33313.2)
Initial guess: [38271.416 33313.162 1000.0 1000.0 0.0 3661.000000 0.0]
Fitted parameters [40497.174 32654.296 2997.8 3596.7 -10.3 4663.985141 -6.3]
Fitted 2 peaks
Trying to fit peak at (50303.2, 38379.8)
Initial guess: [50303.232 38379.803 1000.0 1000.0 0.0 612.000000 0.0]
Fitted parameters [46763.235 34926.801 1246.4 2255.6 -7.0 3647.087119 9.4]
Fitted 3 peaks
Trying to fit peak at (36223.4, 25206.5)
Initial guess: [36223.447 25206.537 1000.0 1000.0 0.0 626.000000 0.0]
Fitted parameters [39426.951 27928.300 1705.4 3167.0 -13.3 2762.323409 3.5]
Fitted 4 peaks
Trying to fit peak at (30847.5, 18873.2)
Initial guess: [30847.529 18873.236 1000.0 1000.0 0.0 271.000000 0.0]
Fitted parameters [32781.370 21577.232 2727.2 2113.7 1.5 385.475753 -0.1]
Fitt

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

## Build phase diagram
This will also find the maximum distance to consider in order to fill a certain amount of the histogram. The amount is set by coverage

*TODO: Check how the distance function works*

*TODO: Make first color (background) black*

In [10]:
coverage = 0.99
phasediagram = registration.phase_diagram(H,xcen,ycen,opts,coverage=coverage)

#plot the phasediagram
cmap = 'Set3'
Y,X = np.meshgrid(ycen,xcen,indexing='ij') #coordinates in dual histogram
plt.figure()
plt.pcolormesh(X,Y,phasediagram,cmap=cmap)
plt.xlabel('X-ray intensity')
plt.ylabel('Neutron intensity')
plt.colorbar()

Finding full coverage: sigma=2, coverage=0.801247354236596
Finding full coverage: sigma=3, coverage=0.8389816810599645
Finding full coverage: sigma=4, coverage=0.8820407797402279
Finding full coverage: sigma=5, coverage=0.9744794868589137
Finding full coverage: sigma=6, coverage=0.9975124357348165


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

## Map from the phase diagram to the volume
This create a segmented volume. It is possible to label just one slice by giving slicenr. slicenr=None will label the full volume.

*TODO Check if this is terribly slow for a real volume*

In [ ]:
%%time
slicenr = None
labels = registration.map_to_volume(x_volume,n_filt,xcen,ycen,phasediagram,slicenr=slicenr)

Labelling slice 0 for 374
Labelling slice 1 for 374
Labelling slice 2 for 374
Labelling slice 3 for 374
Labelling slice 4 for 374
Labelling slice 5 for 374
Labelling slice 6 for 374
Labelling slice 7 for 374
Labelling slice 8 for 374
Labelling slice 9 for 374
Labelling slice 10 for 374
Labelling slice 11 for 374
Labelling slice 12 for 374
Labelling slice 13 for 374
Labelling slice 14 for 374
Labelling slice 15 for 374
Labelling slice 16 for 374
Labelling slice 17 for 374
Labelling slice 18 for 374
Labelling slice 19 for 374
Labelling slice 20 for 374
Labelling slice 21 for 374
Labelling slice 22 for 374
Labelling slice 23 for 374
Labelling slice 24 for 374
Labelling slice 25 for 374
Labelling slice 26 for 374
Labelling slice 27 for 374
Labelling slice 28 for 374
Labelling slice 29 for 374
Labelling slice 30 for 374
Labelling slice 31 for 374
Labelling slice 32 for 374
Labelling slice 33 for 374
Labelling slice 34 for 374
Labelling slice 35 for 374
Labelling slice 36 for 374
Labelling s

In [13]:
plt.figure()
plt.imshow(labels,cmap=cmap)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…